# 目的
違う手法ごとに特定された重みの位置の性質を見る．

具体的には，中間ニューロンのインデックスのユニーク性をチェックしたい．

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from utils.vit_util import transforms_c100, get_batched_hs, get_batched_labels, ViTFromLastLayer, get_ori_model_predictions, identfy_tgt_misclf
from utils.constant import ViTExperiment, Experiment1, Experiment3, ExperimentRepair1, ExperimentRepair2, ExperimentRepair3
from utils.helper import get_device
from utils.de import set_new_weights
from transformers import ViTForImageClassification
from tqdm import tqdm
import torch
import torch.nn as nn
import os
from collections import defaultdict
import numpy as np
import pandas as pd
from datasets import load_from_disk

# デバイスの設定
device = get_device()

def get_location_path(n, w_num, fl_method, location_dir, generate_random=True):
    if fl_method == "ours":
        location_file = f"exp-fl-3_location_n{n}_w{w_num}_weight.npy"
    elif fl_method == "bl":
        location_file = f"exp-fl-2_location_n{n}_weight_bl.npy"
    elif fl_method == "random":
        location_file = f"exp-fl-1_location_n{n}_weight_random.npy"
    else:
        raise ValueError(f"Unknown fl_method: {fl_method}")
    location_path = os.path.join(location_dir, location_file)
    return location_path

Device: cuda


In [30]:
def main(fl_method, n, w_num, rank):
    # rank を tgt_rank として利用（int型）
    tgt_rank = rank
    tgt_pos = ViTExperiment.CLS_IDX
    
    # プリトレーニング済みモデルとキャッシュの hidden states のロード
    pretrained_dir = ViTExperiment.c100.OUTPUT_DIR.format(k=0)
    
    # 実験条件のループ（misclf_type と fpfn は内部ループ）
    misclf_type_list = ["src_tgt", "tgt"] # allは対象外にする
    fpfn_list = [None, "fp", "fn"]
    results_list = []
    for misclf_type in misclf_type_list:
        for fpfn in fpfn_list:
            # ルール：misclf_type=="all" は tgt_rank が 1 かつ fpfn が None のみ有効
            if misclf_type == "all":
                if tgt_rank >= 2 or fpfn is not None:
                    continue
            # misclf_type=="src_tgt" の場合、fpfn は None のみ
            if misclf_type == "src_tgt" and fpfn is not None:
                continue
            location_dir = os.path.join(pretrained_dir, f"misclf_top{tgt_rank}", f"{misclf_type}_weights_location")
            location_path = get_location_path(n, w_num, fl_method, location_dir)
            pos_before, pos_after = np.load(location_path, allow_pickle=True)
            print(f"Location file: {location_path}")
            print(f"pos_before.shape: {pos_before.shape}, pos_after.shape: {pos_after.shape}")
            results_list.append({
                "misclf_type": misclf_type,
                "fpfn": fpfn,
                "pos_before": pos_before,
                "pos_after": pos_after,
            })
    return results_list

In [31]:
all_results = []
fl_method_list = ["ours", "bl", "random"]
tgt_rank_list = [1, 2, 3, 4, 5]

for tgt_rank in tgt_rank_list:
    for fl_method in fl_method_list:
        if fl_method in ["bl", "random"]:
            exp_list = [ExperimentRepair2]
            # exp_list = [Experiment1, ExperimentRepair1, ExperimentRepair2]
        else:
            exp_list = [ExperimentRepair3]
            # exp_list = [Experiment3, ExperimentRepair1, ExperimentRepair2]
        for exp in exp_list:
            if fl_method in ["bl", "random"]:
                n_ratio = exp.NUM_IDENTIFIED_WEIGHTS
                w_num = None
            else:
                n_ratio, w_num = exp.NUM_IDENTIFIED_NEURONS_RATIO, exp.NUM_IDENTIFIED_WEIGHTS
                w_num = 8 * w_num * w_num
            print(f"\nfl_method: {fl_method}, n_ratio: {n_ratio}, w_num: {w_num}, tgt_rank: {tgt_rank}")
            print(f"========================")
            ret_list = main(fl_method=fl_method, n=n_ratio, w_num=w_num, rank=tgt_rank)

            for result in ret_list:
                all_results.append({
                    "fl_method": fl_method,
                    "tgt_rank": tgt_rank,
                    "n_ratio": n_ratio,
                    "w_num": w_num,
                    "misclf_type": result["misclf_type"],
                    "fpfn": result["fpfn"],
                    "pos_before": result["pos_before"],
                    "pos_after": result["pos_after"],
                })



fl_method: ours, n_ratio: 0.12, w_num: 18432, tgt_rank: 1
Location file: /src/src/out_vit_c100_fold0/misclf_top1/src_tgt_weights_location/exp-fl-3_location_n0.12_w18432_weight.npy
pos_before.shape: (8503, 2), pos_after.shape: (9929, 2)
Location file: /src/src/out_vit_c100_fold0/misclf_top1/tgt_weights_location/exp-fl-3_location_n0.12_w18432_weight.npy
pos_before.shape: (10912, 2), pos_after.shape: (7520, 2)
Location file: /src/src/out_vit_c100_fold0/misclf_top1/tgt_weights_location/exp-fl-3_location_n0.12_w18432_weight.npy
pos_before.shape: (10912, 2), pos_after.shape: (7520, 2)
Location file: /src/src/out_vit_c100_fold0/misclf_top1/tgt_weights_location/exp-fl-3_location_n0.12_w18432_weight.npy
pos_before.shape: (10912, 2), pos_after.shape: (7520, 2)

fl_method: bl, n_ratio: 48, w_num: None, tgt_rank: 1
Location file: /src/src/out_vit_c100_fold0/misclf_top1/src_tgt_weights_location/exp-fl-2_location_n48_weight_bl.npy
pos_before.shape: (9042, 2), pos_after.shape: (9390, 2)
Location fil

In [32]:
df_all = pd.DataFrame(all_results)
df_all["misclf_type_name"] = df_all.apply(
    lambda row: row["misclf_type"] if row["fpfn"] is None else f'{row["misclf_type"]}_{row["fpfn"]}',
    axis=1
)

In [42]:
df_all["pos_before"].values[0].shape

(8503, 2)

In [46]:
from collections import defaultdict

# 結果を記録する辞書
summary_before = defaultdict(defaultdict)
summary_after = defaultdict(defaultdict)

# グループ単位で集計
for (misclf_type_name, tgt_rank), group in df_all.groupby(["misclf_type_name", "tgt_rank"]):
    summary_before[misclf_type_name][tgt_rank] = {}
    summary_after[misclf_type_name][tgt_rank] = {}
    for fl_method in group["fl_method"].unique():
        subset = group[group["fl_method"] == fl_method]
        # pos_before の行インデックスを集めてユニーク数をカウント
        all_rows = []
        for arr in subset["pos_before"]:
            arr = np.array(arr)  # 念のため明示
            all_rows.append(arr[:, 0])  # 行方向インデックス（3072次元の方）を抽出
        if all_rows:
            flat = np.concatenate(all_rows)
            flat_unique = np.unique(flat)
            unique_count = len(flat_unique)
        else:
            unique_count = 0
        summary_before[misclf_type_name][tgt_rank][fl_method] = unique_count
        
        # pos_after の行インデックスを集めてユニーク数をカウント
        all_cols = []
        for arr in subset["pos_after"]:
            arr = np.array(arr)
            all_cols.append(arr[:, 1])  # 行方向（W_aft なら 768の方）に変えるならここで調整
        if all_cols:
            flat = np.concatenate(all_cols)
            flat_unique = np.unique(flat)
            unique_count = len(flat_unique)
        else:
            unique_count = 0
        summary_after[misclf_type_name][tgt_rank][fl_method] = unique_count

# 表形式で表示（pandas）
# import pandas as pd
# df_summary = pd.DataFrame(summary).T  # index: (misclf_type, fpfn), columns: fl_method
# df_summary.index.names = ["misclf_type", "fpfn"]

In [47]:
summary_before

defaultdict(collections.defaultdict,
            {'src_tgt': defaultdict(None,
                         {1: {'ours': 119, 'bl': 162, 'random': 192},
                          2: {'ours': 145, 'bl': 176, 'random': 192},
                          3: {'ours': 169, 'bl': 216, 'random': 192},
                          4: {'ours': 157, 'bl': 180, 'random': 192},
                          5: {'ours': 170, 'bl': 188, 'random': 192}}),
             'tgt': defaultdict(None,
                         {1: {'ours': 169, 'bl': 267, 'random': 192},
                          2: {'ours': 215, 'bl': 339, 'random': 192},
                          3: {'ours': 221, 'bl': 312, 'random': 192},
                          4: {'ours': 197, 'bl': 293, 'random': 192},
                          5: {'ours': 213, 'bl': 402, 'random': 192}}),
             'tgt_fn': defaultdict(None,
                         {1: {'ours': 169, 'bl': 267, 'random': 192},
                          2: {'ours': 215, 'bl': 339, 'random': 192}

In [48]:
summary_after

defaultdict(collections.defaultdict,
            {'src_tgt': defaultdict(None,
                         {1: {'ours': 133, 'bl': 163, 'random': 192},
                          2: {'ours': 114, 'bl': 148, 'random': 192},
                          3: {'ours': 125, 'bl': 142, 'random': 192},
                          4: {'ours': 127, 'bl': 171, 'random': 192},
                          5: {'ours': 124, 'bl': 155, 'random': 192}}),
             'tgt': defaultdict(None,
                         {1: {'ours': 125, 'bl': 160, 'random': 192},
                          2: {'ours': 143, 'bl': 191, 'random': 192},
                          3: {'ours': 139, 'bl': 178, 'random': 192},
                          4: {'ours': 129, 'bl': 170, 'random': 192},
                          5: {'ours': 162, 'bl': 206, 'random': 192}}),
             'tgt_fn': defaultdict(None,
                         {1: {'ours': 125, 'bl': 160, 'random': 192},
                          2: {'ours': 143, 'bl': 191, 'random': 192}

# 逆にしてみる

In [49]:
from collections import defaultdict

# 結果を記録する辞書
summary_before = defaultdict(defaultdict)
summary_after = defaultdict(defaultdict)

# グループ単位で集計
for (misclf_type_name, tgt_rank), group in df_all.groupby(["misclf_type_name", "tgt_rank"]):
    summary_before[misclf_type_name][tgt_rank] = {}
    summary_after[misclf_type_name][tgt_rank] = {}
    for fl_method in group["fl_method"].unique():
        subset = group[group["fl_method"] == fl_method]
        # pos_before の行インデックスを集めてユニーク数をカウント
        all_rows = []
        for arr in subset["pos_before"]:
            arr = np.array(arr)  # 念のため明示
            all_rows.append(arr[:, 1])  # 行方向インデックス（3072次元の方）を抽出
        if all_rows:
            flat = np.concatenate(all_rows)
            flat_unique = np.unique(flat)
            unique_count = len(flat_unique)
        else:
            unique_count = 0
        summary_before[misclf_type_name][tgt_rank][fl_method] = unique_count
        
        # pos_after の行インデックスを集めてユニーク数をカウント
        all_cols = []
        for arr in subset["pos_after"]:
            arr = np.array(arr)
            all_cols.append(arr[:, 0])  # 行方向（W_aft なら 768の方）に変えるならここで調整
        if all_cols:
            flat = np.concatenate(all_cols)
            flat_unique = np.unique(flat)
            unique_count = len(flat_unique)
        else:
            unique_count = 0
        summary_after[misclf_type_name][tgt_rank][fl_method] = unique_count

# 表形式で表示（pandas）
# import pandas as pd
# df_summary = pd.DataFrame(summary).T  # index: (misclf_type, fpfn), columns: fl_method
# df_summary.index.names = ["misclf_type", "fpfn"]

In [50]:
summary_before

defaultdict(collections.defaultdict,
            {'src_tgt': defaultdict(None,
                         {1: {'ours': 474, 'bl': 527, 'random': 48},
                          2: {'ours': 476, 'bl': 538, 'random': 48},
                          3: {'ours': 468, 'bl': 533, 'random': 48},
                          4: {'ours': 528, 'bl': 570, 'random': 48},
                          5: {'ours': 468, 'bl': 540, 'random': 48}}),
             'tgt': defaultdict(None,
                         {1: {'ours': 642, 'bl': 683, 'random': 48},
                          2: {'ours': 676, 'bl': 694, 'random': 48},
                          3: {'ours': 625, 'bl': 655, 'random': 48},
                          4: {'ours': 606, 'bl': 612, 'random': 48},
                          5: {'ours': 697, 'bl': 695, 'random': 48}}),
             'tgt_fn': defaultdict(None,
                         {1: {'ours': 642, 'bl': 683, 'random': 48},
                          2: {'ours': 676, 'bl': 694, 'random': 48},
          

In [51]:
summary_after

defaultdict(collections.defaultdict,
            {'src_tgt': defaultdict(None,
                         {1: {'ours': 384, 'bl': 236, 'random': 48},
                          2: {'ours': 285, 'bl': 206, 'random': 48},
                          3: {'ours': 332, 'bl': 229, 'random': 48},
                          4: {'ours': 242, 'bl': 181, 'random': 48},
                          5: {'ours': 301, 'bl': 209, 'random': 48}}),
             'tgt': defaultdict(None,
                         {1: {'ours': 457, 'bl': 352, 'random': 48},
                          2: {'ours': 470, 'bl': 330, 'random': 48},
                          3: {'ours': 401, 'bl': 265, 'random': 48},
                          4: {'ours': 311, 'bl': 223, 'random': 48},
                          5: {'ours': 488, 'bl': 327, 'random': 48}}),
             'tgt_fn': defaultdict(None,
                         {1: {'ours': 457, 'bl': 352, 'random': 48},
                          2: {'ours': 470, 'bl': 330, 'random': 48},
          

In [37]:
from collections import defaultdict

# 結果を記録する辞書
summary_before = defaultdict(dict)
summary_after = defaultdict(dict)

# グループ単位で集計
for misclf_type_name, group in df_all.groupby(["misclf_type_name"]):
    for fl_method in group["fl_method"].unique():
        subset = group[group["fl_method"] == fl_method]
        # pos_before の行インデックスを集めてユニーク数をカウント
        all_rows = []
        for arr in subset["pos_before"]:
            arr = np.array(arr)  # 念のため明示
            all_rows.append(arr[:, 1])  # 行方向インデックス（3072次元の方）を抽出 # XXX: ここが逆！
        if all_rows:
            flat = np.concatenate(all_rows)
            flat_unique = np.unique(flat)
            unique_count = len(flat_unique)
        else:
            unique_count = 0
        summary_before[misclf_type_name][fl_method] = unique_count
        
        # pos_after の行インデックスを集めてユニーク数をカウント
        all_cols = []
        for arr in subset["pos_after"]:
            arr = np.array(arr)
            all_cols.append(arr[:, 0])  # 行方向（W_aft なら 768の方）に変えるならここで調整 # XXX: ここが逆！
        if all_cols:
            flat = np.concatenate(all_cols)
            flat_unique = np.unique(flat)
            unique_count = len(flat_unique)
        else:
            unique_count = 0
        summary_after[misclf_type_name][fl_method] = unique_count


In [38]:
summary_before

defaultdict(dict,
            {'src_tgt': {'ours': 753, 'bl': 758, 'random': 214},
             'tgt': {'ours': 767, 'bl': 768, 'random': 214},
             'tgt_fn': {'ours': 767, 'bl': 768, 'random': 214},
             'tgt_fp': {'ours': 767, 'bl': 768, 'random': 214}})

In [39]:
summary_after

defaultdict(dict,
            {'src_tgt': {'ours': 685, 'bl': 640, 'random': 211},
             'tgt': {'ours': 746, 'bl': 690, 'random': 213},
             'tgt_fn': {'ours': 746, 'bl': 690, 'random': 213},
             'tgt_fp': {'ours': 746, 'bl': 690, 'random': 213}})